# Preprocessing

  1. Open config.py and input the sample names (Training/Testing samples) along with their directories.
  3. Other parameters are as per DeepPathway's implementation. You can change it for your experiments.
  4. Input data files, e.g., pathway database in form of json file or ST metadata csv file should be placed in  Root_DIR.

## Loading Libraries

In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
import cv2
import scipy
import openslide
from PIL import Image
from tiatoolbox import data, logger
import json
from tiatoolbox.wsicore.wsireader import VirtualWSIReader
from tiatoolbox.tools import patchextraction
from huggingface_hub import login
from spatialdata import SpatialData
from spatialdata.models import TableModel, Image2DModel
import torch
import timm
from torchvision import transforms
import os
from pathlib import Path
import config as cfg

|2025-11-18|18:02:29.820| [WARNING] It looks like you are using Pixman version 0.38 (via conda). This version is known to cause issues with OpenSlide. Please consider upgrading to Pixman version 0.39 or later. You may be able do this with the command: conda install -c conda-forge pixman">=0.39"
|2025-11-18|18:02:32.273| [WARNING] /home/e90244aa/anaconda3/envs/bleep_env/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(

|2025-11-18|18:02:32.273| [WARNING] /home/e90244aa/anaconda3/envs/bleep_env/lib/py

## Functions

In [2]:
def get_adata(paths,uni_pathway_genes):
    hvg_bools=[]
    for path in paths:
        adata = sc.read_h5ad(path)
#         adata = adata[adata.obs['in_tissue']==1]
        sc.pp.filter_cells(adata, min_genes=500)
        sc.pp.filter_genes(adata,min_cells=10)
        sc.pp.normalize_total(adata)
        sc.pp.log1p(adata)
        sc.pp.highly_variable_genes(adata, n_top_genes=10000) #for pathways n_top_genes = 10,000
        hvg = adata.var['highly_variable']
        hvg_bools.append(hvg)
    hvg_union = hvg_bools[0]
    for i in range(1, len(paths)):
        print(sum(hvg_union), sum(hvg_bools[i]))
        hvg_union = hvg_union | hvg_bools[i]
    unique_genes= list(hvg_union[hvg_union==True].index.values)
    print("No. of unique genes after filtering",len(unique_genes))
    unique_common_genes = list(set(unique_genes).intersection(set(uni_pathway_genes)))
    print("No. of Common genes between unique pathway genes and unique filtered genes from all samples",len(unique_common_genes))
    return unique_genes
def save_adatas(paths,unique_common_genes):
    filtered_exp_mtxs = []
    barcodes_updated=[]
    for path in paths:
        adata = sc.read_h5ad(path)
        adata = adata[adata.obs['in_tissue']==1]
        sc.pp.filter_cells(adata, min_genes=500)
        sc.pp.normalize_total(adata)
        sc.pp.log1p(adata)
        indices = [i for i, gene in enumerate(adata.var_names.values) if gene in unique_common_genes]
        filtered_exp_mtxs.append(adata[:,indices].X)
        barcodes_updated.append(adata.obs_names)
        print(adata[:, indices].X.shape)
    return filtered_exp_mtxs,barcodes_updated

def check(image,threshold=0.75):
    white=200
    white_pixels = np.sum(np.all(image>=white,axis=-1))
    total = image.shape[0]*image.shape[1]
    percent = white_pixels/total
    return percent>threshold

def check_cropping(samples,image,name,spatial_pos_csv,barcode_csv,res,root_path):
    cropped_images=[]
    white_back_images=[]
    img_filtered_barcodes=[]
    img_file_paths=[]
    parent_dir = root_path + "H&E patches/"  # Change this to your target directory
    # # Create folders
    for folder in samples:
        folder_path = os.path.join(parent_dir, folder)
        os.makedirs(folder_path, exist_ok=True)  # exist_ok=True prevents errors if the folder already exists
    wsi_reader = VirtualWSIReader(image,mpp=(res,res))
    for each, i in zip(barcode_csv,range(len(barcode_csv))):
        v1 =  spatial_pos_csv.loc[spatial_pos_csv[0] == each,4].values[0] #pixel_row_in_full_reS
        v2 =  spatial_pos_csv.loc[spatial_pos_csv[0] == each,5].values[0]  # pixel_col_in_full_res
        patch_extractor = patchextraction.PointsPatchExtractor(input_img=wsi_reader,locations_list=np.array([(v2,v1)]),
                                                               resolution=res,patch_size=(224,224), units='mpp')
        for patch in patch_extractor:
            cropped_image = patch
        if check(cropped_image,0.75):
            white_back_images.append(cropped_image)
        else:
            cropped_images.append(cropped_image)
            img_filtered_barcodes.append(each)
            patch_image = Image.fromarray(cropped_image)
            patch_image.save(parent_dir+name+"/"+each+".png") # saving the spot image into Sample folder.
            img_file_paths.append(parent_dir+name+"/"+each+".png")
    print("More than 75% white background patches",len(white_back_images),"True patches", len(cropped_images))
    return img_filtered_barcodes,img_file_paths

def image_processing(root_path, samples,pixel_res,paths):
    img_fil_paths=[]
    img_fil_barcodes=[]
    for each, res, path in zip(samples, pixel_res,paths):
        print("SAMPLE Name: ", each)
        image_path = root_path + "wsis/" + each + ".tif"
        slide=openslide.OpenSlide(image_path)
        level_dims = slide.level_dimensions[0]
        image = np.array(slide.read_region((0, 0), 0, level_dims).convert("RGB"))
        print("Original Image Shape: ", image.shape)
        print("Pixel Resolution", res)
        spatial_pos_path = root_path +"tissue_position_lists/"+ each + "_tissue_position_list.csv" # if does not exist, please save the sample_name_tissue_position_list.csv into root data directory
        if not Path(spatial_pos_path).exists():
            Path(spatial_pos_path).parent.mkdir(parents=True, exist_ok=True)
            adata = sc.read_h5ad(path)
            temp = adata.obs.iloc[:, 0:5]
            new_order = ['in_tissue', 'array_row', 'array_col','pxl_row_in_fullres','pxl_col_in_fullres']
            temp=temp[new_order]
            temp.to_csv(root_path +"tissue_position_lists/" + each + "_tissue_position_list.csv", header=None,sep=",")
        else:
            pass
        barcode_path = root_path + "filtered_barcodes/"+ each + "_sample_filtered_barcodes.tsv"
        barcode_csv = pd.read_csv(barcode_path, sep="\t", header=None)[0].values
        spatial_pos_csv = pd.read_csv(spatial_pos_path, header=None, sep=",")
        spatial_pos_csv = spatial_pos_csv[spatial_pos_csv[1] == 1]
        img_filtered_barcodes, img_file_paths = check_cropping(samples,image, each, spatial_pos_csv, barcode_csv, res,root_path)
        img_fil_barcodes.append(img_filtered_barcodes)
        img_fil_paths.append(img_file_paths)
    return img_fil_paths,img_fil_barcodes

def filtering_pathway(pathway_dict,unique_common_genes,threshold_pathways=0.70):
    print("Starting Pathway Filtering with 70% threshold")
    filtered_pathway_dic = dict()
    for keys, value in zip(pathway_dict.keys(), pathway_dict.values()):
        intersection = list(set(unique_common_genes).intersection(value))
        if len(intersection) / len(value) >= threshold_pathways and len(intersection) >= 20:
            filtered_pathway_dic[keys] = intersection
    return filtered_pathway_dic

def get_optimus_features(device,root_path,samples):
    # login()  # perform login with your own huggingface key that can be obtained for H-OPtimus Huggingface. Skip login if key is already added.
    model = timm.create_model("hf-hub:bioptimus/H-optimus-0", pretrained=True, init_values=1e-5, dynamic_img_size=False)
    model.to(device)
    model.eval()
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(
            mean=(0.707223, 0.578729, 0.703617),
            std=(0.211883, 0.230117, 0.177517)
        ),
    ])

    with torch.autocast(device_type="cuda", dtype=torch.float16):
        with torch.inference_mode():
            for name in samples:
                feature_vectors = []
                count = 0
                barcodes = pd.read_csv(root_path + "filtered_barcodes/" + name + "_sample_filtered_barcodes_1.tsv", sep="\t", header=None)[0].values
                for bar in barcodes:
                    image = cv2.imread(root_path + "H&E patches/" + name + "/" + bar + ".png")
                    image = transform(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)).unsqueeze(0).to(device)
                    features = model(image)
                    feature_vectors.append(features.cpu().numpy())
                    count += 1
                feature_vectors = np.array(feature_vectors)
                feature_vectors = np.squeeze(feature_vectors, axis=1)
                np.save(root_path + "optimus_features/" + name + "_image_features_optimus-h.npy", feature_vectors)
                print(feature_vectors.shape, "Done")
    return

def save_filtered_barcodes_gene(root_path,paths,barcodes,samples):
    for i in range(0,len(paths)):
        temp=pd.DataFrame(list(barcodes[i]))
        temp.to_csv(root_path+"filtered_barcodes/"+samples[i]+"_sample_filtered_barcodes.tsv",sep="\t",index=False,header=None)
    return
def save_filtered_barcodes_image(root_path,img_fil_barcodes,img_fil_paths,samples):
    print("Final Sample-wise Filtered data shape......")
    for bars, fp, i in zip(img_fil_barcodes, img_fil_paths, samples): # saving the spot-level filtered images and their file paths.
        temp = pd.DataFrame(bars)
        print(i, temp.shape, len(fp))
        temp.to_csv(root_path + "filtered_barcodes/" + i + "_sample_filtered_barcodes_1.tsv", sep="\t", index=False, header=None)
        np.save(root_path + "H&E patches/" + i + "_sample_file_paths.npy", fp)
    return
def save_gene_exp_data(root_path,filtered_exp_mtxs,samples):
    for each, i in zip(filtered_exp_mtxs, range(len(samples))):
        aa = pd.read_csv(root_path + "filtered_barcodes/" + samples[i] + "_sample_filtered_barcodes_1.tsv", sep="\t", header=None)[0].values
        bb = pd.read_csv(root_path + "filtered_barcodes/" + samples[i] + "_sample_filtered_barcodes.tsv", sep="\t", header=None)[0].values
        common_indices = np.intersect1d(aa, bb)
        mask = np.isin(bb, common_indices)
        temp = np.array(each.toarray())[mask]
        print(temp.shape)
        np.save(root_path + "filtered_gene_expression/" + samples[i] + "_sample_unique_genes_pathways_samples.npy", temp)
    return
def create_objects_for_ucell(root_path,samples):
    for each in samples:
        cell_names = pd.read_csv(root_path + "filtered_barcodes/" + each + "_sample_filtered_barcodes_1.tsv",sep="\t", header=None)[0].values
        sample = np.load(root_path + "filtered_gene_expression/" + each + "_sample_unique_genes_pathways_samples.npy").T  # original or predicted sample.
        # print(sample.shape, cell_names.shape)
        gene_names = np.load(root_path+cfg.dataset+'_unique_common_genes.npy',allow_pickle=True)
        scipy.io.savemat(root_path + "data for Ucell calculations/"+ each + "_data.mat",{'x': sample, 'gene_names': gene_names, 'cell_names': cell_names}) # this data will be used in UCell.R code to get the pathway expression matrix of Spots x pathways
    return

def calculate_Rmax_threshold(root_path,samples):
    res=[]
    for sample in samples:
        temp=np.load(root_path+"filtered_gene_expression/"+sample+"_sample_unique_genes_pathways_samples.npy")
        sorted_arr = np.sort(temp, axis=1)[:, ::-1]
        zero_indices = np.argmax(sorted_arr == 0, axis=1)
        for each in zero_indices:
            res.append(each)
#         res.append(np.median(zero_indices))
    print("Rmax Threshold for this data with global median is: ",np.median(res))  # Array of indices where the first 0 appears in each row
    return np.median(res)

    
def get_adata_object(tissue_position_list_data,scalefactor_file_path,data_array,hi_res_image_data,g_names,barcode_names):
    spatial = tissue_position_list_data.loc[tissue_position_list_data[1] == 1, [4, 5]].values
    reversed_array = np.array([[sub_array[1], sub_array[0]] for sub_array in spatial])
    scalefactor_data = scalefactor_file_path
    adata = sc.AnnData(X=data_array)
    if 'spatial' not in adata.uns:
        adata.uns['spatial'] = {}
    if 'C1' not in adata.uns['spatial']:
        adata.uns['spatial']['ST'] = {}

    if 'images' not in adata.uns['spatial']:
        adata.uns['spatial']['ST']['images'] = {}

    if 'scalefactors' not in adata.uns:
        adata.uns['spatial']['ST']['scalefactors'] = {}
    adata.var_names = g_names
    adata.uns['spatial']['ST']['images']['downscaled_fullres'] = hi_res_image_data
    adata.obsm['spatial']= np.array(reversed_array)
    adata.uns['spatial']['ST']['scalefactors']=scalefactor_data
    new_df=pd.DataFrame()
    new_df.index = tissue_position_list_data[0].values
    new_df['in_tissue']=tissue_position_list_data[1].values
    new_df['array_row']=tissue_position_list_data[2].values
    new_df['array_col']=tissue_position_list_data[3].values
    new_df['pxl_row_in_fullres'] = tissue_position_list_data[4].values
    new_df['pxl_col_in_fullres'] = tissue_position_list_data[5].values
    adata.obs=new_df
    return adata

def Sdata_creation(root_path,samples,pathway_dict,dataset='prostate'):
    pathway_names = list(pathway_dict.keys())
    for each in samples:
        adata = sc.read_h5ad(root_path+"st/"+each+".h5ad")
        data_array= np.array(pd.read_csv(root_path+"pathway expression/"+each+"_pathway expression.csv").iloc[:,1:])
        tissue_position_list_data = pd.read_csv(root_path+"tissue_position_lists/"+each+"_tissue_position_list.csv",sep=",",header=None)
        tissue_position_list_data = tissue_position_list_data[tissue_position_list_data[1]==1]
        barcode_data = pd.read_csv(root_path+"filtered_barcodes/"+each+"_sample_filtered_barcodes_1.tsv",sep="\t",header=None)
        res=[]
        for bar in barcode_data[0]:
            temp = list(tissue_position_list_data[tissue_position_list_data[0]==bar].values[0])
            res.append(temp)
        res=pd.DataFrame(res)
        tissue_position_list_data = res
        scalefactor_file_path=adata.uns['spatial']['ST']['scalefactors']
        hi_res_image_data = adata.uns['spatial']['ST']['images']['downscaled_fullres']
        adata_pathway =get_adata_object(tissue_position_list_data,scalefactor_file_path,data_array,hi_res_image_data,pathway_names,barcode_data[0].values)
        optim_feat=np.load(cfg.root_path+"optimus_features/"+each+"_image_features_optimus-h.npy")
        gene_exp = np.load(root_path+"/filtered_gene_expression/"+each+"_sample_unique_genes_pathways_samples.npy")
        adata_optim = sc.AnnData(X=optim_feat)
        adata_optim.obs = adata_pathway.obs
        adata_optim.uns=adata_pathway.uns
        adata_pathway.var_names = pathway_names
        adata_optim.obsm = adata_pathway.obsm
        adata_genes = sc.AnnData(X=gene_exp)
        adata_genes.obs=adata_pathway.obs
        adata_genes.uns=adata_pathway.uns
        adata_genes.obsm= adata_pathway.obsm
        adata_genes.var_names=np.load(root_path+dataset+"_unique_common_genes.npy",allow_pickle=True)
    #     adata_for_sdata = TableModel.parse(adata)
    #     adata_for_sdata=from_legacy_anndata(adata)
        sdata=SpatialData(tables={"adata_pathways":adata_pathway},)
        sdata.tables['optim_feat']=adata_optim
        sdata.tables['filtered_gene_exp']=adata_genes
        sdata.write(root_path+"/SpatialData/"+each+"_spatial_data.zarr",overwrite=True)
    return


## Finding total number of genes in the pathway database

In [3]:
root_path = cfg.root_path
with open(cfg.pathway_dict_file, 'r') as f:
    pathway_dict = json.load(f) # replace with required pathway database, i.e., GO/KEGG/MsigDB
uni_pathway_genes=[]
for value in pathway_dict.values():
    for each in value:
        uni_pathway_genes.append(each)
print("Total Number of unique genes in all pathways: ",len(set(uni_pathway_genes))) # replace with required pathway database, i.e., GO/KEGG/MsigDB

Total Number of unique genes in all pathways:  4383


## Gene Expression Processing

In [4]:
paths=[]
samples=list()
filtered_exp_mtxs=list()
barcodes=list()
samples = cfg.all_samples
for each in samples:
    paths.append(root_path+"st/"+each+".h5ad")
unique_genes= get_adata(paths,uni_pathway_genes)
filtered_exp_mtxs, barcodes = save_adatas(paths,unique_genes)
print("Saving the Filtered Barcodes after spot filtering")
save_filtered_barcodes_gene(root_path, paths, barcodes, samples)

|2025-11-18|18:02:39.723| [WARNING] /home/e90244aa/anaconda3/envs/bleep_env/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)



10000 10000
No. of unique genes after filtering 11866
No. of Common genes between unique pathway genes and unique filtered genes from all samples 2689


|2025-11-18|18:02:40.542| [WARNING] /home/e90244aa/.local/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:176: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["n_genes"] = number

|2025-11-18|18:02:43.045| [WARNING] /home/e90244aa/.local/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:176: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["n_genes"] = number



(4072, 11866)
(3569, 11866)
Saving the Filtered Barcodes after spot filtering


## Image Processing

In [6]:
print("Starting Image Processing......")
pixel_res = cfg.mpp_res  # Image pixel res are obtained from the Metadata file, provided by HEST-1K database
img_fil_paths, img_fil_barcodes = image_processing(root_path, samples, pixel_res, paths)
save_filtered_barcodes_image(root_path, img_fil_barcodes, img_fil_paths, samples)

Starting Image Processing......
SAMPLE Name:  MEND61


|2025-11-18|18:04:23.079| [WARNING] Raw data is None.
|2025-11-18|18:04:23.080| [WARNING] Unknown scale (no objective_power or mpp)


Original Image Shape:  (48128, 47616, 3)
Pixel Resolution 0.1720746926257395
More than 75% white background patches 0 True patches 4072
SAMPLE Name:  MEND62


|2025-11-18|18:06:47.738| [WARNING] Raw data is None.
|2025-11-18|18:06:47.739| [WARNING] Unknown scale (no objective_power or mpp)


Original Image Shape:  (48128, 47616, 3)
Pixel Resolution 0.1721762166660063
More than 75% white background patches 0 True patches 3569
Final Sample-wise Filtered data shape......
MEND61 (4072, 1) 4072
MEND62 (3569, 1) 3569


## Saving unique common genes 

In [10]:
print("Saving the pathway-associated unique common genes")
save_gene_exp_data(root_path, filtered_exp_mtxs, samples)
print("Saving Unique Common Genes:, ", len(unique_genes))
np.save(root_path + cfg.dataset+'_unique_common_genes.npy', unique_genes)

Saving the pathway-associated unique common genes
(4072, 11866)
(3569, 11866)
Saving Unique Common Genes:,  11866


In [11]:
print("Saving the filtered pathway dict in JSON")
filtered_pathway_dic = filtering_pathway(pathway_dict, unique_genes, threshold_pathways=cfg.threshold_pathways) #more than or equal to 70% pathway genes should be in data to calculate pathway expr. .
file_path = root_path + 'pathway_dict_' + str(len(filtered_pathway_dic)) + '.json'
with open(file_path, 'w') as json_file:
    json.dump(filtered_pathway_dic, json_file, indent=4)

Saving the filtered pathway dict in JSON
Starting Pathway Filtering with 70% threshold


## Create Ucell objects for pathway expression calculation

In [12]:
create_objects_for_ucell(root_path, samples)

In [13]:
rmax = calculate_Rmax_threshold(root_path,samples)
print("*********************************************************************************")
print()
print("Please use the following data in UCell.R code to compute Ucell Scores:")
print("Filtered Pathway Dictionary file:",file_path)
print("Samples List:",samples)
print("Data for UCell with path:","./data for Ucell calculations/SAMPLE_ID_data.mat")
print("Rmax=",rmax)
print(f"Ucell Processed Filepath: ./pathway expression/SAMPLE_ID_pathway expression.csv")
print()
print("*********************************************************************************")

Rmax Threshold for this data with global median is:  1543.0
*********************************************************************************

Please use the following data in UCell.R code to compute Ucell Scores:
Filtered Pathway Dictionary file: /home/e90244aa/Bleep/DeepPathwayV2/prostate cancer dataset/pathway_dict_20.json
Samples List: ['MEND61', 'MEND62']
Data for UCell with path: ./data for Ucell calculations/SAMPLE_ID_data.mat
Rmax= 1543.0
Ucell Processed Filepath: ./pathway expression/SAMPLE_ID_pathway expression.csv

*********************************************************************************


- Run Ucell.R code in R Studio to find the Pathway expression.
- Alternatively, you can use python implementation of Ucell, available at: https://pyucell.readthedocs.io/en/latest/index.html#

## Extracting foundation Model features

In [14]:
print("Starting H-Optimus-0 Feature Extraction Process.....") # Obtain login key from Huggingface using https://huggingface.co/bioptimus/H-optimus-0
get_optimus_features(cfg.device, root_path, samples)

Starting H-Optimus-0 Feature Extraction Process.....
|2025-11-18|18:21:48.553| [INFO] Loading pretrained weights from Hugging Face hub (bioptimus/H-optimus-0)
(4072, 1536) Done
(3569, 1536) Done


## Saving processed data as SpatialData Object for fast loading and visualization

In [15]:
print("Saving processed data as SpatialData Objects")
Sdata_creation(root_path,samples,filtered_pathway_dic,dataset=cfg.dataset) #All processed data will be saved to SpatialData object for fast access and processing.

Saving processed data as SpatialData Objects
INFO     The Zarr backing store has been changed from None the new file path:                                      
         /home/e90244aa/Bleep/DeepPathwayV2/prostate cancer dataset/SpatialData/MEND61_spatial_data.zarr           
INFO     The Zarr backing store has been changed from None the new file path:                                      
         /home/e90244aa/Bleep/DeepPathwayV2/prostate cancer dataset/SpatialData/MEND62_spatial_data.zarr           
